## Outils pour la manipulation d'images et librairies.


In [132]:
import PIL as pil
from PIL import Image, ImageTk
import numpy as np
import scipy as sp
import tkinter as tk
from tkinter import filedialog, simpledialog
import os
import random
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

nomImg=""


## Normalisation de l'image (YCbCr et padding)

In [133]:
test = load("test1.png")
Image.fromarray(test,'RGB').show()

QUESTION 1 : 
Donner le code qui transforme une image RGB en une image YCbCr. Vous pourrez produire une matrice pour chaque composante, pour pouvoir plus facilement les manipuler ind ́ependamment. Vous pouvez stocker les donn ́ees YCbCr comme des entiers ou des flottants, mais vous expliquerez votre choix.

In [134]:
def YCbCr(mat):
    matYCbCr=np.empty((mat.shape[0],mat.shape[1],3),dtype=np.float64)

    for i in range(matYCbCr.shape[0]):
        for j in range(matYCbCr.shape[1]):
            matYCbCr[i,j,0] = 0.299 * mat[i,j,0] + 0.587 * mat[i,j,1] + 0.114 * mat[i,j,2]
            
            matYCbCr[i,j,1] = - 0.1687 * mat[i,j,0] - 0.3313 * mat[i,j,1] + 0.5 * mat[i,j,2] + 128

            matYCbCr[i,j,2] = 0.5 * mat[i,j,0] - 0.4187 * mat[i,j,1] - 0.0813 * mat[i,j,2] + 128
            
    return matYCbCr

In [135]:
imgycbcr= YCbCr(test)
Image.fromarray(imgycbcr, 'YCbCr').show()

QUESTION 2 : Donner le code qui transforme une image YCbCr en une image RGB. Attention, les valeurs des canaux RGB doivent ˆetre un entier dans [0, 255] qui pourra ˆetre cod ́e sur un octet. Appliquer successivement la transformation RGB vers YCbCr puis YCbCr vers RGB et v ́erifier que vous obtenez l’image de d ́epart. Vous consulterez la documentation des fonctions de numpy clip, uint8 et mask qui pourraient vous ˆetre utiles.

In [136]:
def RGB(mat):
    matRGB=np.empty([mat.shape[0],mat.shape[1],3],dtype=np.uint8)

    for i in range(matRGB.shape[0]):
        for j in range(matRGB.shape[1]): 
            
            y = mat[i,j,0]
            cb = mat[i,j,1]
            cr = mat[i,j,2]
            
            r = y + 1.402 * (cr - 128) 
            g = y - 0.34414 * (cb - 128) - 0.71414 * (cr - 128)
            b = y + 1.772 * (cb - 128)
            
            matRGB[i,j,0] = int(r)
            
            matRGB[i,j,1] = int(g)
            
            matRGB[i,j,2] = int(b)
             
    return matRGB

In [138]:
imgRGB= RGB(imgycbcr)
Image.fromarray(imgRGB, 'RGB').show()

In [125]:
test = load("test1.png")
compresse = YCbCr(test)
nouveau = RGB(compresse)
print(psnr(test, nouveau))

63.79010503046213


QUESTION 3 : 
Pour pouvoir traiter une image, nous avons besoin que ses dimensions soient des multiples de 8. Pour le garantir, il faut faire du remplissage (padding), c’est `a dire qu’on va ajouter des lignes et des colonnes de pixel noirs en bas et `a droite de l’image. Si on a une image de dimension 15×21, on obtiendra une image de dimension 16 × 24 avec une ligne suppl ́ementaire et trois colonnes suppl ́ementaires. Donner la fonction qui r ́ealise ce padding ainsi que celle qui l’ ́elimine et v ́erifier que l’application de ces deux transformations laissent l’image inchang ́ee.

In [139]:
def padding(mat):
    l = mat.shape[0]
    c = mat.shape[1]
    while l % 8 != 0:
        l += 1
    while c % 8 !=0:
        c += 1
    mat_pad= np.zeros((l, c, 3), dtype=np.uint8)
    mat_pad[:mat.shape[0],:mat.shape[1]]=mat
    return mat_pad
    
def elimine_padding(mat_padding):
    l_pad, c_pad, d = mat.shape
    l = l_pad
    c = c_pad
    while l % 8 != 0:
        l -= 1
    while c % 8 != 0:
        c -=1
    mat_sans_padding=np.empty((l,c,3),dtype=np.uint8)
    for i in range(l):
        for j in range(c):
            mat_sans_padding[i,j] = mat[i,j]
    return mat_sans_padding

In [140]:
imgpadding = padding(test)
Image.fromarray(imgpadding, 'RGB').show()

In [141]:
img_elimine_padding = elimine_padding(imgpadding)
Image.fromarray(img_elimine_padding, 'RGB').show()

QUESTION 4 : L’oeil est moins sensible aux informations de chrominance que de luminance. Pour exploiter cela, nous allons r ́eduire la quantit ́e d’information des canaux Cb et Cr en leur appliquant un sous- ́echantillonnage. Il s’agit de remplacer deux pixels adjacents par la moyenne des deux pixels. Impl ́ementer la fonction qui sous- ́echantillonne une matrice et renvoie une matrice deux fois plus petite.

In [142]:
def petite_mat(mat):
    mat_reduite=np.empty([mat.shape[0], mat.shape[1]//2,3],dtype=np.uint8)
    for i in range(0,mat_reduite.shape[0]):
        for j in range(0,mat_reduite.shape[1]):
            mat_reduite[i,j,1] = (mat[i,2*j,1] + mat[i,2*j+1,1])//2
            mat_reduite[i,j,2] = (mat[i,2*j,2] + mat[i,2*j+1,2])//2
    return mat_reduite


In [156]:
imgreduite  = petite_mat(test)
Image.fromarray(imgreduite, 'YCbCr').show()

/var/folders/83/75fk3cwd5vvb1brl5wypd1dc0000gn/T/ipykernel_53572/3695543901.py:5: RuntimeWarning: overflow encountered in ubyte_scalars
  mat_reduite[i,j,1] = (mat[i,2*j,1] + mat[i,2*j+1,1])//2
/var/folders/83/75fk3cwd5vvb1brl5wypd1dc0000gn/T/ipykernel_53572/3695543901.py:6: RuntimeWarning: overflow encountered in ubyte_scalars
  mat_reduite[i,j,2] = (mat[i,2*j,2] + mat[i,2*j+1,2])//2


QUESTION 5 : Impl ́ementer la fonction qui multiplie par deux la deuxi`eme dimension d’une matrice. Tester `a la suite le sous- ́echantillonnage et cette fonction, vous devez retrouver une image presque identique `a celle de d ́epart.

In [151]:
def matrice_doublee(mat):
    mat_doublee =np.empty([(mat.shape[0]), mat.shape[1]*2, 3], dtype=np.uint8)
    for i in range(mat.shape[0]):
        for j in range(0,mat.shape[1],2):
            mat_doublee[i,j] = mat[i,j//2]
            mat_doublee[i,j+1] = mat[i,j//2]
    return mat_doublee

In [154]:
imgdoublee = matrice_doublee(imgreduite)
Image.fromarray(imgdoublee, 'YCbCr').show()

## Découpage en blocs et compression

QUESTION 6 : Soit une matrice dont les deux dimensions sont divisibles par 8. Donner une fonction qui d ́ecoupe cette matrice en blocs 8 × 8 et les stocke dans une liste. L’ordre des blocs correspond `a l’ordre de lecture d’une image.

In [176]:
matTest = [[2,3,99],[200,8,1],[47,92,179],[200,8,1],[90,3,86],[47,92,179],[200,8,1],[90,3,86],
           [47,92,179],[3,89,190],[200,50,8],[20,30,176],[397,109,19],[47,92,179],[3,89,190],[200,50,8],
           [47,92,179],[49,65,190],[8,79,103],[27,83,39],[49,65,190],[49,65,190],[8,79,103],[27,83,39],
           [47,92,179],[52,49,220],[3,89,190],[20,38,2],[200,50,8],[3,89,190],[20,38,2],[200,50,8],
           [20,30,176],[52,49,220],[2,3,99],[49,65,190],[20,30,176],[20,30,176],[52,49,220],[2,3,99]]


In [181]:
def blocs(mat):
    liste_blocs =[]
    k = 0
    for i in range(0,len(mat),8):
        for j in range (0,len(mat[0]),8) :
            k +=1
            bloc = mat[i:i+8][j:j+8]
            liste_blocs.append(bloc)
    
    #print(liste_blocs)       
    return liste_blocs

#


In [184]:
blocsdécoupe = blocs(matTest)

QUESTION 7 : Les blocs sont d’abord transform ́es en leur repr ́esentation par fr ́equence en utilisant une transform ́ee en cosinus discr`ete. Dans le fichier source projet.ipynb, les fonctions dct2(a) et idct2(a) vous sont fournies. La fonction dct2 donne la repr ́esentation en fr ́equence d’une matrice carr ́ee (qui est une matrice carr ́ee de la mˆeme dimension) et la fonction idct2 calcule la fonction inverse. Donner une fonction qui applique la transform ́ee `a chaque bloc d’une liste.

In [182]:
def transformee(blocs):
    blocs_transformes = []
    for i in range(len(blocs)):
        blocs_transformes.append(dct2(blocs[i]))
    return blocs_transformes

def detransformation(blocs):
    blocs_detransformes = []
    for i in range(len(blocs)):
        blocs_detransformes.append(idct2(blocs[i]))
    return blocs_detransformes

In [185]:
liste_blocs = transformee(blocsdécoupe)
print(liste_blocs)

[array([[ 352.11415053,   61.        ,  155.01854728],
       [ -19.54509246,  -21.57266135,  -14.4178071 ],
       [ -52.08632979,   -7.11770483,    5.36811877],
       [ -22.81322452,  -67.99348818,  -14.43030628],
       [ -78.17954762,   -6.        ,   23.96003617],
       [  -4.82771032,   37.48513792,   -5.95955206],
       [  46.54135346, -256.2846821 ,  -60.96597385],
       [ -12.71767906,  -74.58465179,   -5.98972775]]), array([[ 503.57426629,   -8.        ,   95.84014469],
       [  -9.4740591 ,  -85.54150298,   -8.19758344],
       [ -47.99833777, -111.63557694,  -44.07363748],
       [  79.34258973, -102.70587432,  -17.38392021],
       [  38.17121516,  149.        ,   43.87862046],
       [ -65.52366086, -198.27283483,  -15.37928263],
       [ -16.1320459 ,  119.84363963,    8.25718754],
       [  73.63295589,  258.06789342,   20.98594633]]), array([[ 389.4688691 , -192.25      ,   10.82531755],
       [  48.1990023 ,  -37.20051778,   28.89131014],
       [   3.73381468, 

QUESTION 8 : Vous allez impl ́ementer plusieurs modes de compression. Dans le mode 0, on garde les blocs transform ́es tels quels.
Dans le mode 1, on impose un seuil aux coefficients. Il s’agit de remplacer tous le coefficients plus petit que ce seuil par 0. Vous choisirez un seuil qui permet de supprimer beaucoup de coefficients sans trop d ́egrader l’image. Impl ́ementez le filtrage des coefficients des blocs selon un seuil donn ́e en argument.

In [79]:
def filtrage(blocs,seuil):
    blocs_filtres = []
    for i in range(len(blocs)):
        if blocs[i] < seuil :
            blocs[i] = 0

QUESTION 9 : Dans le mode 2, en plus du filtrage a` seuil, vous appliquerez le sous- ́echantillonnage a` la chrominance (Cb
et Cr). Donner une fonction qui est capable, `a partir d’une image RGB, de cr ́eer les listes de blocs compress ́es dans les 3 modes.

In [94]:
def blocs_compresses(image, mode):
    im = YCbCr(padding(image.all()))
    if mode.all() == 0:
        f = transformee(blocs(im))
    elif mode.all() == 1:
        f = filtrage(transformee(blocs(im)),100)
    elif mode.all() == 2:
        f = petite_mat(filtrage(transformee(blocs(im)),100))
    return f

## Écriture dans un fichier

QUESTION  10 : Pour simplifier cette partie, nous allons  ́ecrire l’information sous forme de texte. Pour ouvrir un fichier en mode  ́ecriture, vous pouvez faire f = open(path, ”w”). Ensuite pour  ́ecrire une ligne vous pouvez faire f.write(”maligne\n”). Si vous avez une variable num ́erique k, pour obtenir la chaˆıne de caract`eres repr ́esentant cette valeur, il suffit de faire str(k).
Les trois questions suivantes d ́etaillent les diff ́erentes parties de votre fonction d’ ́ecriture dans un fichier. Attention `a respecter scrupuleusement les instructions, autrement nous ne pourrons pas lire les fichiers que vous aller produire. Pour commencer, vous  ́ecrirez quatres lignes contenant les informations de votre image. La premi`ere ligne contiendra le type du fichier : “SJPG”. La deuxi`eme ligne contiendra les dimensions de l’image dans l’ordre hauteur puis largeur, s ́epar ́ees par un espace, par exemple “200 300”. La troisi`eme ligne contiendra le mode de compression, par exemple “mode 1”. La quatri`eme ligne contiendra “RLE” si vous utilisez un run length encoding, ou “NORLE” sinon.

In [96]:
mat = load("test1.png")
blocs_compresses(mat,2)

f = open('f','w')

f.write('SJPG\n')
f.write(str(mat.shape[0]))
f.write(' ')
f.write(str(mat.shape[1]))

f.write('\nmode 2')
f.write('\nNORLE')

f.close()

IndexError: tuple index out of range

QUESTION 11 : Vous  ́ecrirez ensuite le contenu des blocs, d’abord ceux de Y, puis ceux de Cb puis ceux de Cr. Chaque bloc est  ́ecrit sur une ligne, les valeurs  ́etant des entiers s ́epar ́es par de espaces.

In [98]:
mat = load("test1.png")
l = blocs_compresses(mat,2)

y =[]
cb = []
cr = []

for i in range(l):
    y.append(int(l[i][0]), ' ')
    cb.append(int(l[i][1]), ' ')
    cr.append(int(l[i][2]), ' ')
    

f = open('f', 'a')
f.write(str(y)) 
f.write('\n')
f.write(str(cb)) 
f.write('\n')
f.write(str(cr)) 
f.write('\n')
f.close()

IndexError: tuple index out of range

QUESTION 12 : Pour am ́eliorer la compression, nous allons utiliser le run length encoding (RLE). Il s’agit d’une m ́ethode similaire au code LZW, mais bien plus simple. Quand un caract`ere est r ́ep ́et ́e plusieurs fois de suite, on va  ́ecrire le caract`ere et son nombre de r ́ep ́etitions. Ici se sont les z ́eros qui sont r ́ep ́et ́es de nombreuses fois, donc on appliquera ce code uniquement aux z ́eros. A` la place de k z ́eros cons ́ecutifs, on  ́ecrira “♯k” ou k est un entier. Ajouter une option `a votre fonction d’ ́ecriture pour qu’elle puisse  ́ecrire les blocs en appliquant le codage RLE.

In [102]:
def rle(texte):
    texte_rle = []
    i = 0
    k = 0

    while i < len(texte):
        if texte[i] != '0':
            texte_rle.append(texte[i])
        elif texte[i] == '0' :
            i += 1
            while texte[i] == '0':
                k += 1
            texte_rle.append('#',k)
        i += 1
    return texte_rle

KeyboardInterrupt: 

## Décompression

QUESTION 13 : E ́crire une fonction de d ́ecompression qui prend une liste de blocs pour chaque canal Y,Cb,Cr contenant les coefficients de la DCT et calcule une matrice repr ́esentant l’image en RGB.

QUESTION 14 : E ́crire une fonction qui lit un fichier SJPG et qui cr ́e ́e les listes de blocs d ́ecrites par le fichier.

## Optimisations

QUESTION 15 :  Impl ́ementer une fonction qui r ́ealise la compression en mode 3. La d ́ecompression ne change pas.

QUESTION 16 : Impl ́ementer la compression et la d ́ecompression par le mode 4. L’ ́ecriture dans un fichier ne change pas.

QUESTION 17 : E ́crire et lire les coefficients dans le fichier en suivant l’ordre en zig-zag.

## Tests 